# Preprocess Interview Dataset

Transform the large Data - Base.csv into a simplified format matching sample-interview-data.csv structure.

In [1]:
import pandas as pd
import numpy as np
import re

# Load the original dataset
df = pd.read_csv(r'c:\Users\khoin\OneDrive - student.vgu.edu.vn\Desktop\VGU\General Study\code\ahp-calculator\public\Data - Base.csv')

print(f"Original dataset shape: {df.shape}")
print(f"\nColumn names:")
print(df.columns.tolist())

Original dataset shape: (21256, 52)

Column names:
['Name', 'Age', 'Gender', 'Type of Graduation/Post Graduation', 'Marital status', 'Mode of interview given by candidate?', 'Pre Interview Check', 'Fluency in English based on introduction', 'Confidence based on Introduction (English)', 'Confidence based on the topic given  ', 'Confidence Based on the PPT Question', 'Confidence based on the sales scenario', 'Structured Thinking (In regional only)', 'Structured Thinking Based on the PPT Question', 'Structured Thinking( Call pitch)', 'Regional fluency based on the topic given  ', 'Regional fluency Based on the PPT Question', 'Regional fluency based on the  sales scenario', 'Does the candidate has mother tongue influence while speaking english.', 'Has acquaintance in Company and has spoken to him/her before applying?', 'Candidate Status', 'Last Fixed CTC (lakhs) ', 'Currently Employed', 'Experienced candidate - (Experience in months)', 'Experienced Candidate (Nature of work)', 'What was th

In [2]:
# Create simplified dataset
processed_df = pd.DataFrame()

# 1. Candidate Name
processed_df['candidate_name'] = df['Name']

# 2. Confidence (average of confidence scores, normalize to 0-10)
# Use existing Confidence Score column
processed_df['confidence'] = df['Confidence Score'] / 12 * 10  # Scale from 0-12 to 0-10

# 3. Communication (based on fluency and regional fluency scores)
processed_df['communication'] = df['Regional Fluency Score'] / 9 * 10  # Scale from 0-9 to 0-10

# 4. Technical Skills (based on structured thinking)
processed_df['technical_skills'] = df['Structured Thinking Score'] / 9 * 10  # Scale from 0-9 to 0-10

# 5. Problem Solving (average of structured thinking aspects)
processed_df['problem_solving'] = df['Structured Thinking Score'] / 9 * 10

# 6. Sales Ability (based on sales scenario confidence)
processed_df['sales_ability'] = df['Confidence based on the sales scenario'].map({
    'Impactful - Good confidence throughout the sales scenario with energy': 9.0,
    'Guarded Confidence - Confident in some areas and ordinary in others': 7.0,
    'Nervous': 5.0,
    'Low confidence and negative energy': 3.0
}).fillna(7.0)

# 7. Experience (convert experience months to a 1-10 scale)
def experience_to_score(exp_text):
    if pd.isna(exp_text) or 'Fresher' in str(exp_text):
        return np.random.uniform(5.5, 6.5)  # Fresher
    elif '6-11.99' in str(exp_text):
        return np.random.uniform(6.5, 7.5)
    elif '12-23.99' in str(exp_text):
        return np.random.uniform(7.0, 8.0)
    elif '24-35.99' in str(exp_text):
        return np.random.uniform(7.5, 8.5)
    elif '36-47.99' in str(exp_text):
        return np.random.uniform(8.0, 9.0)
    elif '48+' in str(exp_text):
        return np.random.uniform(8.5, 9.5)
    else:
        return np.random.uniform(6.0, 7.0)

processed_df['experience'] = df['Experienced candidate - (Experience in months)'].apply(experience_to_score)

# 8. Final Result (map verdict to Selected/Not Selected)
processed_df['final_result'] = df['Interview Verdict'].map({
    'Premium Select': 'Selected',
    'Borderline Select': 'Selected',
    'Select': 'Selected',
    'Reject': 'Not Selected',
    'Borderline Reject': 'Not Selected'
}).fillna('Not Selected')

# Round numerical columns to 1 decimal place
numerical_cols = ['confidence', 'communication', 'technical_skills', 'problem_solving', 'sales_ability', 'experience']
for col in numerical_cols:
    processed_df[col] = processed_df[col].round(1)

print(f"\nProcessed dataset shape: {processed_df.shape}")
print(f"\nFirst 5 rows:")
print(processed_df.head())


Processed dataset shape: (21256, 8)

First 5 rows:
  candidate_name  confidence  communication  technical_skills  \
0         parida         9.2            3.3               7.8   
1         shreej        10.0           10.0              10.0   
2         ms6744         8.3            7.8              10.0   
3         aswalu        10.0           10.0              10.0   
4         aniket         8.3            7.8               5.6   

   problem_solving  sales_ability  experience  final_result  
0              7.8            9.0         7.1  Not Selected  
1             10.0            9.0         6.2  Not Selected  
2             10.0            9.0         5.9      Selected  
3             10.0            9.0         6.0      Selected  
4              5.6            9.0         5.8      Selected  


In [3]:
# Check for missing values
print("Missing values per column:")
print(processed_df.isnull().sum())

print("\n\nData statistics:")
print(processed_df.describe())

print("\n\nFinal result distribution:")
print(processed_df['final_result'].value_counts())

Missing values per column:
candidate_name      0
confidence          0
communication       0
technical_skills    0
problem_solving     0
sales_ability       0
experience          0
final_result        0
dtype: int64


Data statistics:
         confidence  communication  technical_skills  problem_solving  \
count  21256.000000   21256.000000      21256.000000     21256.000000   
mean       6.770662       5.935566          6.893630         6.893630   
std        2.531081       3.271162          3.020813         3.020813   
min        0.800000       0.000000          0.000000         0.000000   
25%        5.800000       3.300000          5.600000         5.600000   
50%        6.700000       5.600000          7.800000         7.800000   
75%        9.200000      10.000000          8.900000         8.900000   
max       10.000000      10.000000         10.000000        10.000000   

       sales_ability    experience  
count   21256.000000  21256.000000  
mean        7.313323      6.39740

In [4]:
# Remove rows with missing values (if any)
processed_df_clean = processed_df.dropna()

# Save to public folder
output_path = r'c:\Users\khoin\OneDrive - student.vgu.edu.vn\Desktop\VGU\General Study\code\ahp-calculator\public\interview-data-processed.csv'
processed_df_clean.to_csv(output_path, index=False)

print(f"\n✅ Processed data saved to: {output_path}")
print(f"Total records: {len(processed_df_clean)}")
print(f"\nSample of saved data:")
print(processed_df_clean.head(10))


✅ Processed data saved to: c:\Users\khoin\OneDrive - student.vgu.edu.vn\Desktop\VGU\General Study\code\ahp-calculator\public\interview-data-processed.csv
Total records: 21256

Sample of saved data:
  candidate_name  confidence  communication  technical_skills  \
0         parida         9.2            3.3               7.8   
1         shreej        10.0           10.0              10.0   
2         ms6744         8.3            7.8              10.0   
3         aswalu        10.0           10.0              10.0   
4         aniket         8.3            7.8               5.6   
5         faizal         5.0            4.4               3.3   
6         ravatn         8.3            7.8               6.7   
7         gornal         6.7            3.3               5.6   
8         Upkarr         5.8            4.4               5.6   
9         moizju         7.5            5.6               8.9   

   problem_solving  sales_ability  experience  final_result  
0              7.8     

In [5]:
# Distribution comparison: Original vs Processed
print("=" * 70)
print("VERDICT DISTRIBUTION COMPARISON")
print("=" * 70)

print("\n📊 ORIGINAL DATASET - 5-Level Verdicts:")
print("-" * 70)
original_verdicts = df['Interview Verdict'].value_counts()
for verdict, count in original_verdicts.items():
    percentage = (count / len(df)) * 100
    print(f"  {verdict:.<40} {count:>6} ({percentage:>5.2f}%)")
print(f"  {'Total':.<40} {len(df):>6} (100.00%)")

print("\n📊 PROCESSED DATASET - 2-Level Classification:")
print("-" * 70)
processed_verdicts = processed_df_clean['final_result'].value_counts()
for verdict, count in processed_verdicts.items():
    percentage = (count / len(processed_df_clean)) * 100
    print(f"  {verdict:.<40} {count:>6} ({percentage:>5.2f}%)")
print(f"  {'Total':.<40} {len(processed_df_clean):>6} (100.00%)")

print("\n✅ Processing Summary:")
print("-" * 70)
print(f"  Original records:        {len(df):>6}")
print(f"  After cleaning:          {len(processed_df_clean):>6}")
print(f"  Records removed:         {len(df) - len(processed_df_clean):>6}")
print("=" * 70)

VERDICT DISTRIBUTION COMPARISON

📊 ORIGINAL DATASET - 5-Level Verdicts:
----------------------------------------------------------------------
  Select..................................   9607 (45.20%)
  Reject..................................   4250 (19.99%)
  Borderline Select.......................   3500 (16.47%)
  Premium Select..........................   1197 ( 5.63%)
  Borderline Reject.......................    150 ( 0.71%)
  Total...................................  21256 (100.00%)

📊 PROCESSED DATASET - 2-Level Classification:
----------------------------------------------------------------------
  Selected................................  14304 (67.29%)
  Not Selected............................   6952 (32.71%)
  Total...................................  21256 (100.00%)

✅ Processing Summary:
----------------------------------------------------------------------
  Original records:         21256
  After cleaning:           21256
  Records removed:              0
